# Brewery Tutorial
__[Open Brewery](https://www.openbrewerydb.org)__ DB is a free dataset and API with public information on breweries, cideries, brewpubs, and bottleshops. The goal of Open Brewery DB is to maintain an open-source, community-driven dataset and provide a public API. Datasets provided by the project are available in the following formats:
- __[CSV](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.csv)__
- __[JSON](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.json)__
- __[PostgreSQL SQL](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.sql)__

For this tutorial, CSV will be used.

TerminusDB Server must be installed on your system before running the Python script. Follow the instructions on __[terminusdb-bootstrap](https://github.com/terminusdb/terminusdb-bootstrap)__. terminusdb-server will be running as a Docker container on http://127.0.0.1:6363.

Python client of TerminusDB is also required. It can be installed from source or through `pip`, you can follow the instructions in the __[repository](https://github.com/terminusdb/terminusdb-client-python)__. When running `pip install terminusdb-client[dataframe]` you will get pandas that is required for reading and import data from CSV files. If installed from source, run `pip install pandas`. tdqm is used for adding a progress bar, run `pip install tqdm` to install it.

## Import libraries
For transfering data from a CSV file to a TerminusDB database, the Python client of TerminusDB and pandas are required. A progress bar is added to the script using tqdm. To import these libraries, add the following lines:

In [ ]:
from typing import List
from terminusdb_client import WOQLQuery, WOQLClient
from terminusdb_client.woqlschema.woql_schema import (
    DocumentTemplate,
    WOQLSchema,
    ValueHashKey,
    HashKey,
)

import pandas as pd
from tqdm import tqdm

## Database management
TerminusDB Server must be installed on your system before running the Python script. Follow the instructions on __[terminusdb-bootstrap](https://github.com/terminusdb/terminusdb-bootstrap)__. terminusdb-server will be running as a Docker container on http://127.0.0.1:6363.

Using the Python client:
- Establish a connection to TerminusDB
- Create a database named *open_brewery*
- Insert schema into database
- Import data from CSV file
- Print data from TerminusDB

In [ ]:
if __name__ == "__main__":
    db_id = "open_brewery"
    url = "https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv"
    client = WOQLClient("http://127.0.0.1:6363")
    client.connect()
    try:
        client.create_database(db_id, accountid="admin", label = "Open Brewery Graph", description = "Create a graph with brewery data")
    except Exception:
        client.set_db(db_id)
    client.insert_document(my_schema.to_dict(),
                           graph_type="schema",
                           commit_msg="I am checking in the schema")
    insert_data(client, url)
    results = client.get_all_documents(graph_type="instance", count=2)
    print("\nRESULTS\n", list(results))

## Schema creation
The dataset has the following columns:
- obdb_id
- name
- brewery_type
- street
- address_2
- address_3
- city
- state
- county_province
- postal_code
- website_url
- phone
- created_at
- updated_at
- country
- longitude
- latitude
- tags

Some of which are optional and rarely have a value assigned and can be omitted when creating the schema and importing the values.

Analyzing the dataset:

- A brewery has *name*, *type*, *address*, *phone* and *website url*
- A brewery can be any of eleven different types (micro, nano, regional, brewpub, large, planning, bar, contract, proprietor, closed)
- An address is a group of values that include *street*, *city*, *postal code* and *coordinates*
- A city is located in a state
- A state is part of a country
- Coordinates are a pair of values, longitude and latitude

Based on what's described above, the following documents are created, each class represents a document in the schema except Brewery Type that is an enum:
- Brewery
- Brewrey_Type
- Address
- City
- State
- Country
- Coordinates

IDs are created using `ValueHashKey` or `HashKey`. `my_schema` is a `WOQLSchema` object that contains the schema itself.

In [ ]:
my_schema = WOQLSchema()

class Coordinates(DocumentTemplate):
    _schema = my_schema
    longitude: float
    latitude: float

class Brewery_Type(EnumTemplate):
    _schema = my_schema
    micro = ()
    nano = ()
    regional = ()
    brewpub = ()
    large = ()
    planning = ()
    bar = ()
    contract = ()
    proprietor = ()
    closed = ()
    taproom = ()

class Country(DocumentTemplate):
    _schema = my_schema
    _key = ValueHashKey()
    name: str

class State(DocumentTemplate):
    _schema = my_schema
    _key = ValueHashKey()
    name: str
    country: Country

class City(DocumentTemplate):
    _schema = my_schema
    _key = ValueHashKey()
    name: str
    state: State

class Address(DocumentTemplate):
    _schema = my_schema
    """This is address"""

    _subdocument = []

    street: str
    city = City
    postal_code: str
    coordinates: List[Coordinates]

class Brewery(DocumentTemplate):
    _schema = my_schema
    name: str
    type_of: Brewery_Type 
    address_of: Address
    phone: str
    website_url: str

## Transfer data
pandas provides built-in functions that make it simple to read and extract data from a CSV file. `read_csv` receives the path of the file as parameter, it can be a URL or a local file. Columns can be specified if not all are required, with `usecols`, `loc` method or `iloc` method.

As some cells in the CSV don't have a value, when importing data it would be required to replace `NULL` values with '' if an integer or float is expected instead. `fillna` must be called to avoid `Not a number` errors.

Before transfering data to TerminusDB, headers and stats of the dataset are printed, all columns in the dataset are included. Output of `df.column.values` and `df.describe(include='all')` is printed to obtain that information. 

Then, only required columns are selected using `loc` method and NULL values are replaced by calling `fillna`.

In [ ]:
def insert_data(client, url):
    all_breweries = []
    df = pd.read_csv(url)
    print("HEADERS\n", list(df.columns.values))
    print("\nSTATS\n", df.describe(include='all'), "\n\nPROGRESS")
    selection = df.loc[:, ['name', 'brewery_type', 'street', 'city', 'state', 'postal_code', 'website_url','phone', 'country', 'longitude', 'latitude']]
    selection = selection.fillna('')

For importing data, a for loop is used to iterate through the values in the CSV, create objects for each document in the schema, assign values to the corresponding variables, and append these values to the `all_breweries` list. A progress bar is added with tqdm.

In [ ]:
    for index, row in tqdm(selection.iterrows(), total=df.shape[0], desc='Reading data'):
        country = Country()
        country.name = row['country']
        state = State()
        state.name = row['state']
        state.country = country
        city = City()
        city.name = row['city']
        city.state = state
        address = Address()
        address.street = row['street']
        address.city = city
        address.postal_code = row['postal_code']
        address.coordinates = [str(row['longitude']), str(row['latitude'])]
        brewery = Brewery()
        brewery.type_of = Brewery_Type[row['brewery_type']]
        brewery.address_of = address
        brewery.phone = row['phone']
        brewery.website_url = row['website_url']
        all_breweries.append(brewery)

Insert the `all_breweries` list into TerminusDB, with ``insert_document`. A progress bar is added with tdqm.

In [ ]:
    with tqdm(total=1, desc='Transfering data') as pbar:
        client.insert_document(all_breweries,
                               commit_msg="Adding all breweries")
        pbar.update(1)